In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import csv
import joblib

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [32]:
data_dir = '../_data/cifar-10-batches-py/'
# img_dir = '../_data/_image/real_img/airplane/'
img_dir = '../_data/_image/truck/'
fileList = os.listdir(data_dir)
fileList = [x for x in fileList]
# fileList = [x for x in fileList if '_batch' in x]
print(fileList)

imgList = os.listdir(img_dir)
imgList = [x for x in imgList]
print(imgList)

['250322.zip', 'batches.meta', 'batch_testing batch 1 of 1', 'batch_training batch 1 of 5', 'batch_training batch 2 of 5', 'batch_training batch 3 of 5', 'batch_training batch 4 of 5', 'batch_training batch 5 of 5', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'result.zip', 'test1.zip', 'testDF.csv', 'testing batch 1 of 1.csv', 'test_batch', 'trainDF.csv', 'training batch 1 of 5.csv', 'training batch 2 of 5.csv', 'training batch 3 of 5.csv', 'training batch 4 of 5.csv', 'training batch 5 of 5.csv']
['truck_ (1).jfif', 'truck_ (1).jpeg', 'truck_ (1).jpg', 'truck_ (10).jfif', 'truck_ (10).jpg', 'truck_ (11).jfif', 'truck_ (11).jpg', 'truck_ (12).jfif', 'truck_ (12).jpg', 'truck_ (13).jfif', 'truck_ (13).jpg', 'truck_ (14).jfif', 'truck_ (14).jpg', 'truck_ (15).jfif', 'truck_ (15).jpg', 'truck_ (16).jfif', 'truck_ (16).jpg', 'truck_ (17).jfif', 'truck_ (17).jpg', 'truck_ (18).jfif', 'truck_ (18).jpg', 'truck_ (19).jfif', 'truck_ (19).jpg',

In [4]:
metadict = unpickle(data_dir+'batches.meta')
answerDict = {}
for i in range(10):
    answerDict[i] = metadict[b'label_names'][i].decode()
print(answerDict)

{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [33]:
real_data = pd.DataFrame(columns=list(range(3072)))
def img32(filename):
    img = cv2.imread(img_dir+filename, cv2.IMREAD_COLOR)
    resized_img = cv2.resize(img, (32,32), interpolation=cv2.INTER_LINEAR)
    real_data.loc[filename] = resized_img.flatten()

In [34]:
for i in imgList:
    img32(i)

In [35]:
real_data

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
truck_ (1).jfif,160,172,200,155,167,195,163,173,189,164,...,104,64,69,94,43,52,81,31,35,60
truck_ (1).jpeg,72,66,55,92,86,81,92,86,79,104,...,71,55,54,52,64,62,61,52,50,49
truck_ (1).jpg,0,1,5,0,1,5,0,1,4,1,...,0,0,0,0,0,0,0,0,0,0
truck_ (10).jfif,254,254,254,255,255,255,255,255,255,255,...,37,58,48,41,52,43,35,55,45,38
truck_ (10).jpg,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
truck_ (7).jpg,0,0,0,0,0,0,0,0,0,0,...,64,44,49,49,0,0,0,0,0,0
truck_ (8).jfif,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
truck_ (8).jpg,220,213,204,223,226,230,223,221,221,222,...,136,114,123,137,130,138,152,120,126,142
truck_ (9).jfif,223,116,1,221,114,0,221,114,0,221,...,196,88,150,190,99,156,198,105,151,187


In [36]:
rf_model = joblib.load("rf_model.joblib")
dt_model = joblib.load("dt_model.joblib")

In [37]:
real_data.columns = real_data.columns+1
# real_data.columns = list(map(str, real_data.columns))
real_data = real_data/255

In [38]:
result_list = []
count = 0
for i in real_data.index:
    # real_pre = dt_model.predict(real_data.loc[i].values.reshape(1, -1))
    real_pre = rf_model.predict(real_data.loc[[i]])
    if i.lower()[:5] == answerDict[real_pre[0]]:
        count+=1
    result_list.append([i, answerDict[real_pre[0]],real_pre[0], count] )
print(result_list)

[['truck_ (1).jfif', 'truck', np.int64(9), 1], ['truck_ (1).jpeg', 'dog', np.int64(5), 1], ['truck_ (1).jpg', 'automobile', np.int64(1), 1], ['truck_ (10).jfif', 'truck', np.int64(9), 2], ['truck_ (10).jpg', 'airplane', np.int64(0), 2], ['truck_ (11).jfif', 'automobile', np.int64(1), 2], ['truck_ (11).jpg', 'automobile', np.int64(1), 2], ['truck_ (12).jfif', 'automobile', np.int64(1), 2], ['truck_ (12).jpg', 'truck', np.int64(9), 3], ['truck_ (13).jfif', 'automobile', np.int64(1), 3], ['truck_ (13).jpg', 'automobile', np.int64(1), 3], ['truck_ (14).jfif', 'ship', np.int64(8), 3], ['truck_ (14).jpg', 'dog', np.int64(5), 3], ['truck_ (15).jfif', 'ship', np.int64(8), 3], ['truck_ (15).jpg', 'truck', np.int64(9), 4], ['truck_ (16).jfif', 'automobile', np.int64(1), 4], ['truck_ (16).jpg', 'automobile', np.int64(1), 4], ['truck_ (17).jfif', 'frog', np.int64(6), 4], ['truck_ (17).jpg', 'automobile', np.int64(1), 4], ['truck_ (18).jfif', 'automobile', np.int64(1), 4], ['truck_ (18).jpg', 'airp

In [39]:
real_data.shape

(72, 3072)

In [42]:
count = 0
for i in real_data.index:
    # real_pre = dt_model.predict(real_data.loc[i].values.reshape(1, -1))
    real_pre = rf_model.predict(real_data.loc[[i]])
    if i.lower()[:5] == answerDict[real_pre[0]]:
        count+=1
    print(i, answerDict[real_pre[0]],real_pre[0] )
    print(count)

truck_ (1).jfif truck 9
1
truck_ (1).jpeg dog 5
1
truck_ (1).jpg automobile 1
1
truck_ (10).jfif truck 9
2
truck_ (10).jpg airplane 0
2
truck_ (11).jfif automobile 1
2
truck_ (11).jpg automobile 1
2
truck_ (12).jfif automobile 1
2
truck_ (12).jpg truck 9
3
truck_ (13).jfif automobile 1
3
truck_ (13).jpg automobile 1
3
truck_ (14).jfif ship 8
3
truck_ (14).jpg dog 5
3
truck_ (15).jfif ship 8
3
truck_ (15).jpg truck 9
4
truck_ (16).jfif automobile 1
4
truck_ (16).jpg automobile 1
4
truck_ (17).jfif frog 6
4
truck_ (17).jpg automobile 1
4
truck_ (18).jfif automobile 1
4
truck_ (18).jpg airplane 0
4
truck_ (19).jfif automobile 1
4
truck_ (19).jpg dog 5
4
truck_ (2).jfif automobile 1
4
truck_ (2).jpeg automobile 1
4
truck_ (2).jpg automobile 1
4
truck_ (20).jfif automobile 1
4
truck_ (20).jpg deer 4
4
truck_ (21).jfif automobile 1
4
truck_ (21).jpg automobile 1
4
truck_ (22).jfif automobile 1
4
truck_ (22).jpg automobile 1
4
truck_ (23).jfif automobile 1
4
truck_ (23).jpg truck 9
5
truck_ (